# Sense HAT for PYNQ:Character display

This notebook illustrates how to interact with the [Sense HAT](https://www.raspberrypi.org/products/sense-hat/) and display character on the LED matrix of Sense HAT.


This example notebook does the following.
1. import python libraries
2. select rpi switch and using MicroblazeLibrary
3. configure the I2C device
4. convert characters
5. waitting user's input and display on the Sense HAT LED matrix

![PYNQ with Sense HAT](data/PYNQ_with_Sense_HAT.jpg)


### 1. Sense HAT Introduction

The Sense HAT, which is a fundamental part of the [Astro Pi](https://astro-pi.org/) mission, allows your board to sense the world around it.I thas an 8×8 RGB LED matrix, a five-button joystick and includes the following sensors:

* Gyroscope
* Accelerometer
* Magnetometer
* Temperature
* Barometric pressure
* Humidity

![Sense HAT add-on board](data/Sense_HAT.jpg)


### 2. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to
RPI header (by default, the pins will be connected to PMODA instead).

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from PIL import Image
import time
import numpy as np

In [2]:
base = BaseOverlay('base.bit')
base.select_rpi()
lib = MicroblazeLibrary(base.RPI, ['i2c','xio_switch','circular_buffer'])

### 3. Configure the I2C device
Initialize the I2C device and set the I2C pin of RPI header 

In [3]:
device = lib.i2c_open_device(1)
lib.set_pin(2, lib.SDA1)
lib.set_pin(3, lib.SCL1)

In [4]:
LED_MATRIX_ADDRESS = 0x46

### 4. Convert characters
Convert characters to pixels

In [5]:
img = Image.open("./data/sense_hat_text.png").convert('RGB')
text = open("./data/sense_hat_text.txt", 'r').read()
text_pixels = list(map(list, img.getdata()))

In [6]:
text_dict = {}
for index, s in enumerate(text):
    start = index * 40
    end = start + 40
    char = text_pixels[start:end]
    text_dict[s] = char

### 5. Display
Wait user's input and display on the LED matrix

In [7]:
buf = bytearray(193)
buf[0] = 0

In [8]:
true = 1
while true:
    display = input("Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminal.\n")
    if(display == "c" or display == "C" ):
        true = 0
    else:
        display = display + " "
        for value in display:
            for i in range(0,193) :
                buf[i] = 0;
            for y in range(1,8) :
                for x in range(2,7) :
                    buf[1+x+8*0+3*8*y] = int(text_dict[value][8*(6-x)+(y-1)][0]/20);
                    buf[1+x+8*1+3*8*y] = 0;
                    buf[1+x+8*2+3*8*y] = 0;
            lib.i2c_write(device, LED_MATRIX_ADDRESS, buf, 193)
            time.sleep(1)

Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminal.
HELLO,XILINX!
Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminal.
C


### 6.Clean up
Close the I2C device and switch back the connection on the shared pin to PMODA header.

In [9]:
device.close()
base.select_pmoda()